In [74]:
import pandas as pd
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter

In [46]:
spotify2023 = pd.read_excel("spotify-2023.xlsx",sheet_name="spotify-2023")

spotify2023.loc[:, 'major'] = [0 if mde=='minor' else 1 for mde in spotify2023['mode'].values]
spotify2023.loc[:, 'minor'] = [0 if mde=='major' else 1 for mde in spotify2023['mode'].values]
spotify2023.loc[:, 'spotify_charts_flag'] = np.abs(spotify2023.not_in_spotify_charts.values - 1)
print(spotify2023.describe())
print(spotify2023.columns)


       artist_count  released_year  released_month  released_day  \
count    953.000000     953.000000      953.000000    953.000000   
mean       1.556139    2018.238195        6.033578     13.930745   
min        1.000000    1930.000000        1.000000      1.000000   
25%        1.000000    2020.000000        3.000000      6.000000   
50%        1.000000    2022.000000        6.000000     13.000000   
75%        2.000000    2022.000000        9.000000     22.000000   
max        8.000000    2023.000000       12.000000     31.000000   
std        0.893044      11.116218        3.566435      9.201949   

       in_spotify_playlists  in_spotify_charts  in_apple_playlists  \
count            953.000000         953.000000          953.000000   
mean            5200.124869          12.009444           67.812172   
min               31.000000           0.000000            0.000000   
25%              875.000000           0.000000           13.000000   
50%             2224.000000          

In [5]:
RandomizedSearchCV?


Init signature:
RandomizedSearchCV(
    estimator,
    param_distributions,
    *,
    n_iter=10,
    scoring=None,
    n_jobs=None,
    refit=True,
    cv=None,
    verbose=0,
    pre_dispatch='2*n_jobs',
    random_state=None,
    error_score=nan,
    return_train_score=False,
)
Docstring:     
Randomized search on hyper parameters.

RandomizedSearchCV implements a "fit" and a "score" method.
It also implements "score_samples", "predict", "predict_proba",
"decision_function", "transform" and "inverse_transform" if they are
implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized
by cross-validated search over parameter settings.

In contrast to GridSearchCV, not all parameter values are tried out, but
rather a fixed number of parameter settings is sampled from the specified
distributions. The number of parameter settings that are tried is
given by n_iter.

If all parameters are presented as a list,
sampling without replacement is p

In [6]:
RandomForestClassifier?

Init signature:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None,
)
Docstring:     
A random forest classifier.

A random forest is a meta estimator that fits a number of decision tree
classifiers on various sub-samples of the dataset and uses averaging to
improve the predictive accuracy and control over-fitting.
Trees in the forest use the best split strategy, i.e. equivalent to passing
`splitter="best"` to the underlying :class:`~sklearn.tree.DecisionTreeRegressor`.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the 

In [9]:
for col in [
    'not_in_apple_charts', 'not_in_apple_playlists',
    'not_in_deezer_charts', 'not_in_deezer_playlists',
    'not_in_spotify_charts', 'not_in_spotify_playlists'
]:
    
    print(col, Counter(spotify2023[col]))


not_in_apple_charts Counter({0: 853, 1: 101})
not_in_apple_playlists Counter({0: 930, 1: 24})
not_in_deezer_charts Counter({1: 559, 0: 395})
not_in_deezer_playlists Counter({0: 929, 1: 25})
not_in_spotify_charts Counter({0: 548, 1: 406})
not_in_spotify_playlists Counter({0: 953, 1: 1})


### spotify charts will act as label for classifier

In [61]:
feat_cols = [
    'bpm',
    'danceability_%', 
    'valence_%', 
    'energy_%',
    'acousticness_%', 
    'instrumentalness_%', 
    'liveness_%', 
    'speechiness_%',
    'spotify_charts_flag'
]

feats_df = spotify2023[feat_cols]
print(feats_df.isna().sum(axis=0))
feats_df = feats_df.dropna()
label_col = feats_df['spotify_charts_flag'].values
feats_df = feats_df[feats_df.columns[:-1]].values
print(feats_df)

bpm                    1
danceability_%         1
valence_%              1
energy_%               1
acousticness_%         1
instrumentalness_%     1
liveness_%             1
speechiness_%          1
spotify_charts_flag    0
dtype: int64
[[130.  65.  49. ...   4.   7.   5.]
 [ 96.  23.  19. ...   0.  40.   3.]
 [139.  36.  22. ...   0.  11.   4.]
 ...
 [140.  56.  48. ...   0.  35.   4.]
 [125.  80.  89. ...   0.   8.   4.]
 [128.  86.  42. ...   0.   9.  19.]]


### creating param grid for randomized CV search

In [62]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees in the forest
    'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'sqrt', 'log2'],   # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30, 40, 50],    # Maximum number of levels in each tree
    'min_samples_split': np.arange(2, 50, 2).astype('int').tolist(),            # Minimum number of samples required to split a node
    'min_samples_leaf': np.arange(1, 50, 2).astype('int').tolist(),              # Minimum number of samples required at each leaf node
    'bootstrap': [True, False],                 # Method of selecting samples for training each tree
    'criterion': ['gini', 'entropy', 'log_loss']            # Function to measure the quality of a split
}

### running random grid search with random forest and cross-validation

In [63]:
X_train, X_test, y_train, y_test = train_test_split(feats_df, label_col, test_size=0.2, random_state=42)

In [64]:
random_cv = RandomizedSearchCV(RandomForestClassifier(), param_grid, cv=5, n_iter=1000, verbose=1)
random_cv.fit(X_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=1000,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [None, 10, 20, 30, 40, 50],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 3, 5, 7, 9, 11,
                                                             13, 15, 17, 19, 21,
                                                             23, 25, 27, 29, 31,
                                                             33, 35, 37, 39, 41,
                                                             43, 45, 47, 49],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              12, 14, 16, 18,
                                                              20, 22, 24, 26,
                                                              28, 30, 32, 34,
                                                              36, 38, 40, 42,
                                                              44, 46, 48],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   verbose=1)

In [69]:
print(random_cv.best_estimator_.score(X_test, y_test))
for ii, imp in enumerate(random_cv.best_estimator_.feature_importances_):
    print(f"{feat_cols[ii]} importance: {imp:0.4f}")

0.612565445026178
bpm importance: 0.1870
danceability_% importance: 0.1254
valence_% importance: 0.1555
energy_% importance: 0.1086
acousticness_% importance: 0.1742
instrumentalness_% importance: 0.0093
liveness_% importance: 0.1377
speechiness_% importance: 0.1023


In [70]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of boosting stages to perform
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],  # Shrinks the contribution of each tree by learning_rate
    'max_depth': np.arange(3, 20, 1).astype('int').tolist(),  # Maximum depth of the individual regression estimators
    'min_samples_split': np.arange(2, 50, 1).astype('int').tolist(),  # The minimum number of samples required to split an internal node
    'min_samples_leaf': np.arange(1, 50, 1).astype('int').tolist(),  # The minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2', None],  # The number of features to consider when looking for the best split
    'subsample': [0.8, 0.9, 1.0],  # The fraction of samples to be used for fitting the individual base learners
    'loss': ['deviance', 'exponential']  # Loss function to be optimized ('deviance' for logistic regression, 'exponential' for AdaBoost-like behavior)
}

In [71]:
random_gbcv = RandomizedSearchCV(GradientBoostingClassifier(), param_grid, cv=5, n_iter=10000, verbose=1)
random_gbcv.fit(X_train, y_train)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


/home/lreclusa/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
24605 fits failed out of a total of 50000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24605 fits failed with the following error:
Traceback (most recent call last):
  File "/home/lreclusa/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/lreclusa/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/lreclusa/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_params
    va

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_iter=10000,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.15,
                                                          0.2, 0.25],
                                        'loss': ['deviance', 'exponential'],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      11, 12, 13, 14, 15, 16,
                                                      17, 18, 19],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10, 11, 12,
                                                              13, 14, 15, 16,
                                                              17, 18, 19, 20,
                                                              21, 22, 23, 24,
                                                              25, 26, 27, 28,
                                                              29, 30, 31, ...],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.8, 0.9, 1.0]},
                   verbose=1)

In [73]:
print(random_gbcv.score(X_test, y_test))
for ii, imp in enumerate(random_gbcv.best_estimator_.feature_importances_):
    print(f"{feat_cols[ii]} importance: {imp:0.4f}")

0.6230366492146597
bpm importance: 0.1689
danceability_% importance: 0.1196
valence_% importance: 0.1722
energy_% importance: 0.1498
acousticness_% importance: 0.1583
instrumentalness_% importance: 0.0118
liveness_% importance: 0.1221
speechiness_% importance: 0.0972


In [60]:
fig,axs = plt.subplots(

array([0.19732339, 0.        , 0.        , 0.10267122, 0.20093216,
       0.11692683, 0.15325402, 0.00601682, 0.13046692, 0.09240865])